In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely import wkb
from shapely import errors
import itertools
import matplotlib.pyplot as plt
import datetime as dt
import holidays
import math
import ast

from Resources import ml_utils as mlu
from Resources import ml_utils2 as mlu2

* notebook to run master functions to impute model ready dfs

In [2]:
weather_df = pd.read_csv('../Resources/NewYorkCity_Temperature')

In [3]:
zone_long_lat_data = pd.read_parquet('https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones_4326.parquet')

In [4]:
coords_df = pd.read_csv('../Resources/zone_coords_ids.csv')
coords_df['coords'] = coords_df['coords'].apply(ast.literal_eval)
coords_df[['latitude', 'longitude']] = pd.DataFrame(coords_df['coords'].tolist(), index=coords_df.index)
coords_df = coords_df[['LocationID', 'Zone', 'Borough', 'latitude', 'longitude']]
coords_df.head()

,LocationID,Zone,Borough,latitude,longitude
0,1,Newark Airport,EWR,40.689064,-74.177255
1,2,Jamaica Bay,Queens,40.603994,-73.835412
2,3,Allerton/Pelham Gardens,Bronx,40.865430,-73.867365
3,4,Alphabet City,Manhattan,40.725102,-73.979583
4,5,Arden Heights,Staten Island,40.563700,-74.191603


In [5]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')
mar_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')
apr_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet')
may_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet')
jun_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet')
jul_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet')
aug_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet')
sep_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet')
oct_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet')
nov_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-11.parquet')
dec_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-12.parquet')

In [6]:
dfs = [jan_df, feb_df, mar_df, apr_df, may_df, jun_df, jul_df, aug_df, sep_df, oct_df, nov_df, dec_df]

ny_taxi_2024_df = pd.concat(dfs)

In [7]:
jan_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet').sample(frac=0.035, random_state=1)
feb_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet').sample(frac=0.035, random_state=1)
mar_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet').sample(frac=0.035, random_state=1)
apr_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet').sample(frac=0.035, random_state=1)
may_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet').sample(frac=0.035, random_state=1)
jun_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-06.parquet').sample(frac=0.035, random_state=1)
jul_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet').sample(frac=0.035, random_state=1)
aug_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-08.parquet').sample(frac=0.035, random_state=1)
sep_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-09.parquet').sample(frac=0.035, random_state=1)
oct_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-10.parquet').sample(frac=0.035, random_state=1)
nov_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-11.parquet').sample(frac=0.035, random_state=1)
dec_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-12.parquet').sample(frac=0.035, random_state=1)

In [8]:
fhv_dfs = [jan_app_df, feb_app_df, mar_app_df, apr_app_df, mar_app_df, jun_app_df, jul_app_df, aug_app_df, sep_app_df, oct_app_df, nov_app_df, dec_app_df]
fhv_2024_df = pd.concat(fhv_dfs)

In [9]:
ny_taxi_2024_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [ ]:
# uses centraid values for PU and DO x, y's
results = mlu.impute_all(ny_taxi_2024_df, fhv_2024_df, zone_long_lat_data, weather_df)

results

c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fhv_2024_df['trip_distance'] = round(fhv_2024_df['trip_distance'], 2)
c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils.py:147: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 16.11  14.3   22.36 ...  53.6   56.68 147.16]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fhv_2024_df.loc[calc, 'total_amount'] = sum_of_columns[calc]


,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,...,False,False,False,True,False,False,True,False,False,False


In [11]:
pickup_coords = coords_df[['LocationID', 'latitude', 'longitude']].copy()
pickup_coords

,LocationID,latitude,longitude
0,1,40.689064,-74.177255
1,2,40.603994,-73.835412
2,3,40.865430,-73.867365
3,4,40.725102,-73.979583
4,5,40.563700,-74.191603
...,...,...,...
258,259,40.886272,-73.878581
259,260,40.745380,-73.905415
260,261,40.711900,-74.012527
261,262,40.768311,-73.964406


In [12]:
results.columns

MultiIndex([(   'second_of_day',),
            (     'day_of_year',),
            (         'weekend',),
            (         'holiday',),
            (    'morning_rush',),
            (    'evening rush',),
            (            'prcp',),
            (            'temp',),
            (             'PUx',),
            (             'PUy',),
            (             'DOx',),
            (             'DOy',),
            (        'distance',),
            (   'duration(sec)',),
            (            'fare',),
            (           'tolls',),
            (         'airport',),
            (      'congestion',),
            (           'total',),
            (           'class',),
            (        'PU_Bronx',),
            (     'PU_Brooklyn',),
            (    'PU_Manhattan',),
            (       'PU_Queens',),
            ('PU_Staten Island',),
            (        'DO_Bronx',),
            (     'DO_Brooklyn',),
            (    'DO_Manhattan',),
            (       

In [13]:
for column in results:
    print(results[column].value_counts())

(second_of_day,)
36000    308
27000    304
32400    297
28800    297
30600    270
        ... 
16068      6
15399      6
12903      5
15191      5
18417      5
Name: count, Length: 86400, dtype: int64
(day_of_year,)
69     38329
62     36701
83     35544
61     32588
68     32497
       ...  
141     4975
126     4974
146     4265
147     4048
148     3874
Name: count, Length: 366, dtype: int64
(weekend,)
False    4209501
True     1834946
Name: count, dtype: int64
(holiday,)
0    5904561
1     139886
Name: count, dtype: int64
(morning_rush,)
0    5409901
1     634546
Name: count, dtype: int64
(evening rush,)
0    5277744
1     766703
Name: count, dtype: int64
(prcp,)
0.0    5283658
0.2     118544
0.1      87385
0.3      78634
0.5      67790
        ...   
4.6        307
6.0        284
4.0        281
7.4        212
5.5          7
Name: count, Length: 72, dtype: int64
(temp,)
44.06    146504
41.00    143846
42.08    138963
48.02    133471
46.04    132671
          ...  
60.62       134
6

In [14]:
#results.to_parquet('Resources/model_df.parquet')

In [ ]:
# (WILL use lat and long coords for PU and DO x, y's) for now, keeps ID numbers for merge with lat long zone data
results2 = mlu2.impute_all(ny_taxi_2024_df, fhv_2024_df, zone_long_lat_data, weather_df)

results2

c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils2.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fhv_2024_df['trip_distance'] = round(fhv_2024_df['trip_distance'], 2)
c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils2.py:147: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 16.11  14.3   22.36 ...  53.6   56.68 147.16]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fhv_2024_df.loc[calc, 'total_amount'] = sum_of_columns[calc]


,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,PULocationID,DOLocationID,prcp,temp,distance,duration(sec),fare,tolls,airport,congestion,total,class
0,5,1,False,1,0,0,132,41,0.0,42.80,18.85,1599.0,70.00,6.94,1.75,0.0,95.88,0
1,9,1,False,1,0,0,3,51,0.0,42.80,2.47,785.0,14.59,0.00,0.00,0.0,14.59,1
2,33,1,False,1,0,0,145,112,0.0,42.80,2.02,474.0,20.62,0.00,0.00,0.0,20.62,2
3,38,1,False,1,0,0,16,73,0.0,42.80,2.47,459.0,23.20,0.00,0.00,0.0,23.20,2
4,42,1,False,1,0,0,164,161,0.0,42.80,1.06,335.0,7.90,0.00,0.00,2.5,15.48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971769,85846,366,False,0,0,0,58,213,0.0,48.02,2.48,534.0,14.38,0.00,0.00,0.0,14.38,2
5971770,85864,366,False,0,0,0,189,62,0.0,48.02,1.58,488.0,15.01,0.00,0.00,0.0,15.01,2
5971771,85870,366,False,0,0,0,79,79,0.0,48.02,0.60,256.0,5.80,0.00,0.00,2.5,10.80,0
5971772,86042,366,False,0,0,0,180,63,0.0,48.02,1.36,266.0,8.92,0.00,0.00,0.0,8.92,1


In [16]:
for column in results2:
    print(results2[column].value_counts())

(second_of_day,)
27000    304
36000    299
28800    291
32400    285
30600    267
        ... 
15399      6
16068      6
12903      5
15191      5
18417      5
Name: count, Length: 86400, dtype: int64
(day_of_year,)
69     37833
62     36098
83     35069
61     32137
68     32072
       ...  
126     4974
141     4970
146     4259
147     4045
148     3872
Name: count, Length: 366, dtype: int64
(weekend,)
False    4161122
True     1810652
Name: count, dtype: int64
(holiday,)
0    5833924
1     137850
Name: count, dtype: int64
(morning_rush,)
0    5346033
1     625741
Name: count, dtype: int64
(evening rush,)
0    5212234
1     759540
Name: count, dtype: int64
(PULocationID,)
132    156307
138    141016
161    137764
237    126886
236    113690
        ...  
99         73
2          12
110         8
199         2
105         1
Name: count, Length: 261, dtype: int64
(DOLocationID,)
236    119011
161    117456
138    117229
237    116482
132    113038
        ...  
111       179
99       

In [ ]:
#results2.to_parquet('Resources/model_df_2.parquet')